<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-CatBoost" data-toc-modified-id="Модель-CatBoost-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель CatBoost</a></span></li><li><span><a href="#Модель-LightGBM" data-toc-modified-id="Модель-LightGBM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель LightGBM</a></span></li><li><span><a href="#Модель-Logistic-Regression" data-toc-modified-id="Модель-Logistic-Regression-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель Logistic Regression</a></span></li><li><span><a href="#Результаты-обучения" data-toc-modified-id="Результаты-обучения-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Результаты обучения</a></span></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Классификация комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Требуется обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении имеется набор данных с разметкой о токсичности правок.

<b>Цель</b>: Построить модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

In [1]:
#импорт библиотек
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

from catboost import CatBoostClassifier

import lightgbm as ltb

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

import re

from tqdm import notebook
from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv("/datasets/toxic_comments.csv") #загрузка датасета

In [3]:
data.info() #общая информация о данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Оценим баланс классов:

In [4]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Видно, что количество объектов класса 0 значительно больше. Учтем это в дальнейшем при обучении.

In [5]:
pd.set_option('max_colwidth', 400)
data.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do ...",0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [6]:
lemmatizer = WordNetLemmatizer() #инициализируем лемматизатор

Функции лемматизации и очистки текста:

In [7]:
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

def clear_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    return ' '.join(text.split()) 

Применяем функции к датасету:

In [8]:
tqdm.pandas()
data['lemm_text'] = data['text'].progress_apply(lambda x: lemmatize(clear_text(x)))

  0%|          | 0/159571 [00:00<?, ?it/s]

In [9]:
STATE = np.random.RandomState(12345) #объект RandomState для дальнейшего использования по всему проекту

Разбиваем датасет на тренировочную, валидационную и тестовую выборки в соотношении 3-1-1. Так как векторы строк зависят от полного набора текстов, то проводить кросс-валидацию проблематично (нужно каждый раз переопределять векторы). Поэтому используем валидационную выборку.

In [10]:
corpus_total = data['lemm_text'].values
target_total = data['toxic']

features_train, features_part, target_train, target_part = train_test_split(
    corpus_total, target_total, test_size=0.4, random_state=STATE)

features_test, features_valid, target_test, target_valid = train_test_split(
    features_part, target_part, test_size=0.5, random_state=STATE)

Проводим векторизацию текстов в полученных выборках:

In [11]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words = stopwords) 
tf_idf_train = count_tf_idf.fit_transform(features_train) 
print('Размер матрицы tf-idf_train:', tf_idf_train.shape)

Размер матрицы tf-idf_train: (95742, 117494)


In [12]:
tf_idf_valid = count_tf_idf.transform(features_valid) 
print('Размер матрицы tf-idf_valid:', tf_idf_valid.shape)

Размер матрицы tf-idf_valid: (31915, 117494)


In [13]:
tf_idf_test = count_tf_idf.transform(features_test) 
print('Размер матрицы tf-idf_test:', tf_idf_test.shape)

Размер матрицы tf-idf_test: (31914, 117494)


<b>Вывод: </b>данные подготовлены - получена общая информация, датасет разбит на выборки, проведена векторизация тестов, размерности TF-IDF матриц в порядке.

## Обучение

Функция для обучения моделей, возвращающая результаты по метрике F1:

In [14]:
summary = pd.DataFrame() #инициализируем датафрейм для записи результатов обучения моделей

def model_run(model, features_train, target_train, features_valid, target_valid, summary):
    model.fit(features_train, target_train)
    
    F1_train = f1_score(target_train, model.predict(features_train))
    F1_valid = f1_score(target_valid, model.predict(features_valid))
    
    summary = summary.append({'model' : type(model).__name__,
                            'F1_train' : F1_train,
                            'F1_valid' : F1_valid} , ignore_index=True)
    return summary

Для учета дисбаланса создадим словарь весов `dict_classes`, который используем для параметра `class_weight`:

In [15]:
class_ratio = data['toxic'].value_counts()[0] / data['toxic'].value_counts()[1]
class_weights={0:1, 1:class_ratio}

### Модель CatBoost

In [17]:
%%time

summary = model_run(CatBoostClassifier(iterations=200, eval_metric='F1', verbose=10),tf_idf_train, target_train, tf_idf_valid, target_valid, summary)

Learning rate set to 0.316097
0:	learn: 0.4006517	total: 2.35s	remaining: 7m 46s
10:	learn: 0.5941772	total: 20.1s	remaining: 5m 44s
20:	learn: 0.6211481	total: 37.1s	remaining: 5m 16s
30:	learn: 0.6508557	total: 53.6s	remaining: 4m 52s
40:	learn: 0.6834105	total: 1m 10s	remaining: 4m 32s
50:	learn: 0.6986708	total: 1m 26s	remaining: 4m 13s
60:	learn: 0.7134034	total: 1m 43s	remaining: 3m 55s
70:	learn: 0.7307548	total: 1m 59s	remaining: 3m 37s
80:	learn: 0.7392547	total: 2m 16s	remaining: 3m 19s
90:	learn: 0.7461543	total: 2m 32s	remaining: 3m 2s
100:	learn: 0.7544257	total: 2m 48s	remaining: 2m 45s
110:	learn: 0.7604313	total: 3m 4s	remaining: 2m 28s
120:	learn: 0.7656553	total: 3m 21s	remaining: 2m 11s
130:	learn: 0.7690725	total: 3m 37s	remaining: 1m 54s
140:	learn: 0.7715528	total: 3m 53s	remaining: 1m 37s
150:	learn: 0.7742363	total: 4m 9s	remaining: 1m 20s
160:	learn: 0.7770969	total: 4m 25s	remaining: 1m 4s
170:	learn: 0.7793782	total: 4m 41s	remaining: 47.7s
180:	learn: 0.7820

### Модель LightGBM

In [18]:
%%time
summary = model_run(ltb.LGBMClassifier(max_depth=10, learning_rate=0.5, class_weight=class_weights, n_jobs=-1),
                    tf_idf_train, target_train, tf_idf_valid, target_valid, summary)

CPU times: user 1min 45s, sys: 410 ms, total: 1min 45s
Wall time: 1min 46s


### Модель Logistic Regression

In [19]:
%%time

lr = LogisticRegression(solver='liblinear', class_weight=class_weights, C=10)

summary = model_run(lr, tf_idf_train, target_train, tf_idf_valid, target_valid, summary)

CPU times: user 9.71 s, sys: 22 s, total: 31.7 s
Wall time: 31.7 s


### Результаты обучения

In [20]:
display(summary)

,F1_train,F1_valid,model
0,0.789963,0.742145,CatBoostClassifier
1,0.851680,0.740401,LGBMClassifier
2,0.938293,0.753687,LogisticRegression


Таким образом, лучшей моделью на валидационной выборке по метрике F1 стала Logistic Regression.

### Тестирование лучшей модели

In [21]:
print('F1 лучшей модели на тестовой выборке:', f1_score(target_test, lr.predict(tf_idf_test)))

F1 лучшей модели на тестовой выборке: 0.7628383321141186


## Выводы

Таким образом, было исследовано несколько моделей, классифицирющих токсичные и нетоксичные комментарии. В процессе работы признаки были подготовлены с помощью TF-IDF метода, а затем на полученных признаках было обучено три модели:

    *CatBoost
    *LightGBM
    *Logistic Regression

Лучшей моделью на валидационной выборке стала Logistic Regression. Эта модель была исследована на тестовой выборке, требуемое значение F1-метрики 0.75 было достигнуто.